<a href="https://colab.research.google.com/github/ElenaBetelgeuse/Text-processing/blob/main/%D0%A0%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%D1%81_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Задача:** реализуйте задачу классификации на основе BERT-like модели и KNN на данных Russian Intents Dataset с Kaggle.


Создаем классификаторы текстов в условиях большого числа маленьких классов, состоящих из коротких текстов.


**Результат:** код для создания поискового векторного индекса + логика определения класса на основе близости к обучающим объектам (по ближайшему, по топ-N ближайших, и т. п.).

In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'qa-intents-dataset-university-domain:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1549969%2F2560015%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240523%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240523T061952Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5d3a8d3eaf589152e899679a879cafbd8d3bd20aa414dc786c469a2c8eb1240e25ac27990789b80fe315e17ad83c1c894354a130b6fceeb7afc1e5ee8f9f9dccadcc98bb4f8686827a3aae57eeab4ffa640f6b9d32387f859aea636b905d13e2da3c9617409a9bb7eefd4f330ba1bc710f11af3da124c108ae79607501b3d2661a6d0d7151521cc09c37d00a9a58ebfb5330ca8305736ffd621eeb38804e53d9f54214d142aab75a44b0bb8a04f7588ec39759c5a4323a1a9551d3df0633330543eac96159bab49054984681c824c254e291ec8da9a7f715f762e67bbaad297d80df6e26a97f0e3e21a332e96629a657cd49de42ab4c3ea626ff4b459d6a4393'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 100575 bytes downloaded
Downloaded and uncompressed: qa-intents-dataset-university-domain
Data source import complete.


In [ ]:
import numpy as np
import pandas as pd
import torch
import transformers

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-xlm-r-multilingual-v1')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-xlm-r-multilingual-v1')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
train = pd.read_table(
    '/kaggle/input/qa-intents-dataset-university-domain/dataset_train.tsv',
    header=None,names=['txt','c']).head(10000) # на полных данных у меня села оперативка, я решила ограничить трейновые данные до 10 000 значений

# создадим в трейновом датафрейме новый столбец с числовым значением каждого класса (используя поиск уникальных значений по столбцу класса)
cats=train.c.unique()
icats = {w: i for i, w in enumerate(cats)}
train["ic"]=train.c.map(icats)


test = pd.read_table(
    '/kaggle/input/qa-intents-dataset-university-domain/dataset_test.tsv',
    header=None,names=['txt','c'])


In [ ]:
train.head()

,txt,c,ic
0,мне нужна справка,statement_general,0
1,оформить справку,statement_general,0
2,взять справку,statement_general,0
3,справку как получить,statement_general,0
4,справку ммф где получаться,statement_general,0


In [ ]:
test.head()

,txt,c
0,как получить справку,statement_general
1,мне нужна справка,statement_general
2,справка студента эф петь,conform
3,справка студента фф оформлять,conform
4,как мне заказать справка об обучении,conform


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   txt     10000 non-null  object
 1   c       10000 non-null  object
 2   ic      10000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 234.5+ KB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883 entries, 0 to 882
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   txt     883 non-null    object
 1   c       883 non-null    object
dtypes: object(2)
memory usage: 13.9+ KB


In [ ]:
# создаём токены из столбца с текстами
treintoken=tokenizer(train['txt'].tolist(),padding=True, truncation=True, return_tensors='pt')
testtoken=tokenizer(test['txt'].tolist(),padding=True, truncation=True, return_tensors='pt')

In [ ]:
# создадим список из числовых классов
ids=train['ic'].tolist()

In [ ]:
# создадим функцию для вычисления среднего пуллинга
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output['last_hidden_state']
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [ ]:
# создадим эмбеддинги из токенов
with torch.no_grad():
    treinembeding=mean_pooling(model(**treintoken),treintoken['attention_mask'])
    testembeding=mean_pooling(model(**testtoken),testtoken['attention_mask'])

In [ ]:
!pip install nmslib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Created wheel for nmslib: filename=nmslib-2.1.1-cp310-cp310-linux_x86_64.whl size=13578846 sha256=bcedfcd9f6456b85e7c3ab02920ff2de2c23109db00b86552ac35b409b9c5b57
  Stored in directory: /root/.cache/pip/wheels/21/1a/5d/4cc754a5b1a88405cad184b76f823897a63a8d19afcd4b9314
Successfully built nmslib


In [ ]:
import nmslib
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(treinembeding) #,ids=ids)
index.createIndex({'post': 2}, print_progress=True)

In [ ]:
# выведем k=5 ближайших соседей для 15 рандомных классов
for n in np.random.randint(0,test.shape[0],15):
    id,dist=index.knnQuery(testembeding[n],k=5)
    print(n, test.iloc[n].txt,test.iloc[n].c)
    for i,d in zip(id,dist):
        print(d,train.iloc[i].txt,train.iloc[i].c)
    print()

564 находится 20187 sched_for_group
0.13131404 ср расписание 207194 sched_for_group_day
0.1429975 завтра расписание 219717 группы sched_for_group_day
0.14617121 вторник расписание 203190 sched_for_group_day
0.14889681 пятница 19927 группа sched_for_group_day
0.15055645 ср расписание 204586 группы sched_for_group_day

846 объединение студенческий нгу расположен где гдепомоги studentorgs_define
0.17289656 создавать объединение студенческий нгу student_union_create
0.1785397 осуществить объединение студенческий нгу student_union_create
0.19619125 сформировать объединение студенческий нгу student_union_create
0.19747615 организация студенческая создавать student_union_create
0.20230138 создавать объединение студенческий student_union_create

555 расписание группы 22532 sched_for_group
0.11463845 расписание группы 223405 sched_for_group
0.13911992 вс 22512 расписание пар находится sched_for_group_day
0.1523217 расписание у 22844 группы sched_for_group
0.16839498 вторник 22745 группа sched_f